## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Tefe,BR,-3.3542,-64.7114,86.00,84,75,8.05,broken clouds
1,1,Faanui,PF,-16.4833,-151.7500,81.99,67,27,4.81,scattered clouds
2,2,Georgetown,MY,5.4112,100.3354,82.40,57,20,3.44,few clouds
3,3,Mataura,NZ,-46.1927,168.8643,54.00,84,93,4.00,overcast clouds
4,4,Vaini,TO,-21.2000,-175.2000,75.20,94,20,2.30,light rain


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 95


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Tefe,BR,-3.3542,-64.7114,86.00,84,75,8.05,broken clouds
1,1,Faanui,PF,-16.4833,-151.7500,81.99,67,27,4.81,scattered clouds
2,2,Georgetown,MY,5.4112,100.3354,82.40,57,20,3.44,few clouds
4,4,Vaini,TO,-21.2000,-175.2000,75.20,94,20,2.30,light rain
6,6,Cayenne,GF,4.9333,-52.3333,87.80,55,40,12.66,scattered clouds
7,7,Rikitea,PF,-23.1203,-134.9692,80.33,76,100,12.71,moderate rain
11,11,Carnarvon,AU,-24.8667,113.6333,78.80,69,0,10.36,clear sky
15,15,Kununurra,AU,-15.7667,128.7333,75.20,100,100,6.91,moderate rain
21,21,Busselton,AU,-33.6500,115.3333,70.66,55,89,6.91,overcast clouds
23,23,Arraial Do Cabo,BR,-22.9661,-42.0278,89.60,65,20,10.36,few clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                294
City                   294
Country                291
Lat                    294
Lng                    294
Max Temp               294
Humidity               294
Cloudiness             294
Wind Speed             294
Current Description    294
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_preferred_cities_df = preferred_cities_df.dropna()

In [7]:
clean_preferred_cities_df.count()

City_ID                291
City                   291
Country                291
Lat                    291
Lng                    291
Max Temp               291
Humidity               291
Cloudiness             291
Wind Speed             291
Current Description    291
dtype: int64

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(15)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Tefe,BR,86.00,broken clouds,-3.3542,-64.7114,
1,Faanui,PF,81.99,scattered clouds,-16.4833,-151.7500,
2,Georgetown,MY,82.40,few clouds,5.4112,100.3354,
4,Vaini,TO,75.20,light rain,-21.2000,-175.2000,
6,Cayenne,GF,87.80,scattered clouds,4.9333,-52.3333,
7,Rikitea,PF,80.33,moderate rain,-23.1203,-134.9692,
11,Carnarvon,AU,78.80,clear sky,-24.8667,113.6333,
15,Kununurra,AU,75.20,moderate rain,-15.7667,128.7333,
21,Busselton,AU,70.66,overcast clouds,-33.6500,115.3333,
23,Arraial Do Cabo,BR,89.60,few clouds,-22.9661,-42.0278,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [10]:
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Tefe,BR,86.00,broken clouds,-3.3542,-64.7114,Aracati Hotel
1,Faanui,PF,81.99,scattered clouds,-16.4833,-151.7500,Bora-Bora Pearl Beach Resort & Spa
2,Georgetown,MY,82.40,few clouds,5.4112,100.3354,Cititel Penang
4,Vaini,TO,75.20,light rain,-21.2000,-175.2000,Keleti Beach Resort
6,Cayenne,GF,87.80,scattered clouds,4.9333,-52.3333,Hôtel Le Dronmi
7,Rikitea,PF,80.33,moderate rain,-23.1203,-134.9692,Pension Maro'i
11,Carnarvon,AU,78.80,clear sky,-24.8667,113.6333,Hospitality Carnarvon
15,Kununurra,AU,75.20,moderate rain,-15.7667,128.7333,Hotel Kununurra
21,Busselton,AU,70.66,overcast clouds,-33.6500,115.3333,Observatory Guest House
23,Arraial Do Cabo,BR,89.60,few clouds,-22.9661,-42.0278,Pousada Porto Praia


In [11]:
hotel_df.count()

City                   291
Country                291
Max Temp               291
Current Description    291
Lat                    291
Lng                    291
Hotel Name             291
dtype: int64

In [17]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df[hotel_df["Hotel Name"] != ""]
clean_hotel_df.count()

City                   277
Country                277
Max Temp               277
Current Description    277
Lat                    277
Lng                    277
Hotel Name             277
dtype: int64

In [18]:
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Tefe,BR,86.00,broken clouds,-3.3542,-64.7114,Aracati Hotel
1,Faanui,PF,81.99,scattered clouds,-16.4833,-151.7500,Bora-Bora Pearl Beach Resort & Spa
2,Georgetown,MY,82.40,few clouds,5.4112,100.3354,Cititel Penang
4,Vaini,TO,75.20,light rain,-21.2000,-175.2000,Keleti Beach Resort
6,Cayenne,GF,87.80,scattered clouds,4.9333,-52.3333,Hôtel Le Dronmi
7,Rikitea,PF,80.33,moderate rain,-23.1203,-134.9692,Pension Maro'i
11,Carnarvon,AU,78.80,clear sky,-24.8667,113.6333,Hospitality Carnarvon
15,Kununurra,AU,75.20,moderate rain,-15.7667,128.7333,Hotel Kununurra
21,Busselton,AU,70.66,overcast clouds,-33.6500,115.3333,Observatory Guest House
23,Arraial Do Cabo,BR,89.60,few clouds,-22.9661,-42.0278,Pousada Porto Praia


In [19]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [20]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [21]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))